In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.9 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 67.4 MB/s 
     |████████████████████████████████| 3.3 MB 71.4 MB/s 
     |████████████████████████████████| 596 kB 87.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import *
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import metrics

from tensorflow.keras.applications.inception_v3 import InceptionV3
from itertools import combinations as comb
import ntpath
import glob
import math
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import OneHotEncoder 
from transformers import TFBertModel, DistilBertTokenizerFast


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/"

/content/drive/My Drive


In [ ]:
data = pd.read_json('./archive/Sarcasm_Headlines_Dataset.json', lines= True)

In [ ]:
data

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [ ]:
X = data['headline']
y = data['is_sarcastic']

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,do_lower_case = True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

#Train test split
train_sents,test_sents, train_labels, test_labels  = train_test_split(X,y,test_size=0.15)

train_ids = encoder(train_sents)
test_ids = encoder(test_sents) 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid',kernel_regularizer='l2')(dense)    

model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x = train_ids, y = train_labels, epochs = 5, verbose = 1, batch_size = 64, validation_data = (test_ids, test_labels))

Epoch 1/5
355/355 [==============================] - 72s 161ms/step - loss: 2.4611 - accuracy: 0.8523 - val_loss: 2.2931 - val_accuracy: 0.8912
Epoch 2/5
355/355 [==============================] - 55s 154ms/step - loss: 2.1454 - accuracy: 0.9264 - val_loss: 2.0776 - val_accuracy: 0.9189
Epoch 3/5
355/355 [==============================] - 55s 154ms/step - loss: 1.9150 - accuracy: 0.9551 - val_loss: 1.9412 - val_accuracy: 0.9186
Epoch 4/5
355/355 [==============================] - 55s 154ms/step - loss: 1.7120 - accuracy: 0.9745 - val_loss: 1.8065 - val_accuracy: 0.9244
Epoch 5/5
355/355 [==============================] - 55s 154ms/step - loss: 1.5350 - accuracy: 0.9834 - val_loss: 1.7042 - val_accuracy: 0.9156


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [ ]:
bert_encoder = Model(input_word_ids, model.get_layer('lambda').output)
bert_encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [ ]:
train_embed = bert_encoder.predict(train_ids)
test_embed = bert_encoder.predict(test_ids)

In [ ]:
train_embed.shape

(22702, 768)

In [ ]:
def generate_imp_space(X_train, Y_train, X_test, imp_feature_size, foz):
  X_train=pd.DataFrame(X_train)
  X_test=pd.DataFrame(X_test)
  imp_train_data = X_train.values
  imp_test_data = X_test.values
  Y_train=pd.DataFrame(Y_train)
  d = len(X_train.columns)
  num_class=len(np.unique(Y_train))
  # print("____1_____")
  for i in range(0,imp_feature_size*foz):
    Xindis = np.random.permutation(d)
    for j in range(0,d-(foz-1),foz):  #d/foz kadar doner
      sX = np.random.permutation(num_class)
      s1 = sX[0]
      # print("____2_____")
      s1data = X_train[X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s2data = X_train[~X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s1data = s1data.iloc[:,Xindis[j:j+(foz)]]
      s2data = s2data.iloc[:,Xindis[j:j+(foz)]] # s1 vs all other classes, #foz feature
      # print("____3_____")
      s1label = np.ones((s1data.values.shape[0],1),dtype=int)
      s2label = -1*np.ones((s2data.values.shape[0],1),dtype=int)
      Wdata = np.concatenate((s1data,s2data))
      # print("____4_____")
      
      Wdata = x2fx(Wdata)
      Wlabel = np.concatenate((s1label,s2label))
      W = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Wdata.T, Wdata)),Wdata.T),Wlabel)
      
      WW = x2fx(X_train.iloc[:,Xindis[j:j+(foz)]].values)
      imp_train_data = np.concatenate((imp_train_data, np.matmul(WW,W)),axis=1)
      
      TT = x2fx(X_test.iloc[:,Xindis[j:j+(foz)]].values)
      imp_test_data = np.concatenate((imp_test_data, np.matmul(TT,W)),axis=1)
    
  return imp_train_data,imp_test_data
  

In [ ]:
def x2fx(x, model='quadratic'):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == 'linear':
        return linear
    if model == 'purequadratic':
        return np.c_[linear, x**2]
    interaction = np.array([x[:,i]*x[:,j] for i, j in comb(range(x.shape[1]), 2)]).T
    if model == 'interaction':
        return np.c_[linear, interaction]
    if model == 'quadratic':
        return np.c_[linear, interaction, x**2]
def MajorityVoting(votes):
  results = []
  for i in range(0,votes.shape[1]):
    values, counts = np.unique(votes[:,i], return_counts=True)
    results.append(values[np.argmax(counts)])
  return np.array(results)

In [ ]:
foz=4
imp_feature_size=1
acc_imp=[]

acc_norm=[]
acc_incept=[]
imp_tree_predicts=[]
tree_predicts=[]
train_embed=pd.DataFrame(train_embed)
d=len(train_embed.columns)
for i in range(0,5):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imp_feature_size, foz)
    imp_d = imp_tr.shape[1]
    imp_sel_d = 2* round(math.log2(imp_d)) #feature
    sel_d = 2*round(math.log2(d))
      
    imp_tree = GradientBoostingClassifier(max_features=imp_sel_d, n_estimators=50,random_state=42)
   
    imp_tree.fit(imp_tr, train_labels)
    imp_tree_predicts.append(imp_tree.predict(imp_ts))


    tree = GradientBoostingClassifier(max_features=sel_d, n_estimators=50,random_state=42)
    
    tree.fit(train_embed, train_labels)
    tree_predicts.append(tree.predict(test_embed))
result_imp=MajorityVoting(np.array(imp_tree_predicts))
result_norm=MajorityVoting(np.array(tree_predicts))
acc_imp.append(metrics.accuracy_score(test_labels, result_imp))
acc_norm.append(metrics.accuracy_score(test_labels, result_norm))
#result_norm=encoder.fit_transform(result_norm)
#result_imp=encoder.fit_transform(result_imp)

In [ ]:
print("gb")
print(acc_norm)
print(acc_imp)

gb
[0.9266284002994759]
[0.922635388070876]


In [ ]:
print(acc_norm)
print(acc_imp)

[0.9308709757923633]
[0.9293735962066384]


In [ ]:
imp_tr.shape

(22702, 1536)

In [ ]:
result_imp=MajorityVoting(np.array(imp_tree_predicts))
print(x)
metrics.accuracy_score(test_labels, result_imp)

0.9303718492637884